In [1]:
import pandas as pd
import numpy as np

In [2]:
import sklearn.datasets as skds
from pathlib import Path

In [3]:
np.random.seed(1237) # For reproducibility

In [4]:
#http://qwone.com/~jason/20Newsgroups/
path_train = "C:\\Users\\dasarred\\20news-bydate-train"

In [5]:
files_train = skds.load_files(path_train,load_content=False)

In [6]:
label_index = files_train.target
label_names = files_train.target_names
labelled_files = files_train.filenames

In [7]:
data_tags = ["filename","category","news"]

In [8]:
data_list = []

In [9]:
i=0
for f in labelled_files:
    data_list.append((f,label_names[label_index[i]],Path(f).read_text()))
    i += 1

In [10]:
print(data_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
data = pd.DataFrame.from_records(data_list, columns=data_tags)

In [12]:
# lets take 80% data as training and remaining 20% for test.
train_size = int(len(data) * .8)
 
train_posts = data['news'][:train_size]
train_tags = data['category'][:train_size]
train_files_names = data['filename'][:train_size]
 
test_posts = data['news'][train_size:]
test_tags = data['category'][train_size:]
test_files_names = data['filename'][train_size:]

In [13]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer

ModuleNotFoundError: No module named 'keras'

In [15]:
# select markdown and use below cmd to display image in notebook itself , keep image in local directory 
#<img src="keras_installation.PNG">

<img src="keras_installation.png">

In [18]:
# open anaconda prompt from windows start and use below cmd 
#pip install keras
import keras
print('keras: %s' % keras.__version__)
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer

keras: 2.2.0


In [20]:
# 20 news groups
num_labels = 20
vocab_size = 15000
batch_size = 100
 
# define Tokenizer with Vocab Size
# tokenizer(texts_to_matrix) used to convert your text into a numeric vector
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_posts)
 
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [21]:
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [22]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               7680512   
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                10260     
__________

In [23]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])
 
text_labels = encoder.classes_
 
for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction[0])]
    print(test_files_names.iloc[i])
    print('Actual label:' + test_tags.iloc[i])
    print("Predicted label: " + predicted_label)

2263/2263 [==============================] - 1s 318us/step
Test accuracy: 0.8784798956581096
C:\Users\dasarred\20news-bydate-train\alt.atheism\53114
Actual label:alt.atheism
Predicted label: alt.atheism
C:\Users\dasarred\20news-bydate-train\comp.graphics\38666
Actual label:comp.graphics
Predicted label: comp.graphics
C:\Users\dasarred\20news-bydate-train\sci.med\58932
Actual label:sci.med
Predicted label: sci.med
C:\Users\dasarred\20news-bydate-train\sci.crypt\15212
Actual label:sci.crypt
Predicted label: sci.crypt
C:\Users\dasarred\20news-bydate-train\comp.os.ms-windows.misc\9695
Actual label:comp.os.ms-windows.misc
Predicted label: comp.os.ms-windows.misc
C:\Users\dasarred\20news-bydate-train\rec.sport.baseball\104482
Actual label:rec.sport.baseball
Predicted label: rec.sport.baseball
C:\Users\dasarred\20news-bydate-train\soc.religion.christian\20731
Actual label:soc.religion.christian
Predicted label: sci.med
C:\Users\dasarred\20news-bydate-train\comp.graphics\38583
Actual label:com

In [24]:
# creates a HDF5 file 'my_model.h5'
model.model.save('my_model.h5')

import pickle

# Save Tokenizer i.e. Vocabulary
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

C:\Users\dasarred\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [25]:
encoder.classes_ #LabelBinarizer

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype='<U24')

In [26]:
# These are the labels we stored from our training
# The order is very important here.
 
labels = np.array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x',
 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball',
 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space',
 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast',
 'talk.politics.misc', 'talk.religion.misc'])
 
test_files = ["C:\\Users\\dasarred\\20news-bydate-test\\comp.graphics\\38758",
              "C:\\Users\\dasarred\\20news-bydate-test\\misc.forsale\\76115",
              "C:\\Users\\dasarred\\20news-bydate-test\\soc.religion.christian\\21329"
              ]
x_data = []
for t_f in test_files:
    t_f_data = Path(t_f).read_text()
    x_data.append(t_f_data)
 
x_data_series = pd.Series(x_data)
x_tokenized = tokenizer.texts_to_matrix(x_data_series, mode='tfidf')
 
i=0
for x_t in x_tokenized:
    prediction = model.predict(np.array([x_t]))
    predicted_label = labels[np.argmax(prediction[0])]
    print("File ->", test_files[i], "Predicted label: " + predicted_label)
    i += 1

File -> C:\Users\dasarred\20news-bydate-test\comp.graphics\38758 Predicted label: comp.graphics
File -> C:\Users\dasarred\20news-bydate-test\misc.forsale\76115 Predicted label: comp.sys.ibm.pc.hardware
File -> C:\Users\dasarred\20news-bydate-test\soc.religion.christian\21329 Predicted label: soc.religion.christian
